# Capstone Project - Battle of the Neighborhoods 
## IBM Data Science Professional Certificate


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)
* [Recommendation](#recommendation)


## Introduction: Business Problem <a name="introduction"></a>

The goal of this project is to find the optimal location to start a Fitness-Restaurant, named Fit and Food, in New York City. This place will combine a gym with a restaurant where tasty, healthy food is the staple. It is meant to be a place for people to go to eat after working out, grabbing a quick healthy lunch as well as a place to order take-away. 

The optimal location will be determined by looking at the neighborhoods of NYC in a number of ways. It is important that the neighborhood is not crowded with restaurants. On top of that it is important that the neighborhood in which to start Fit and Food has a health-centered attitude. This is because health-minded individuals are more likely to make use of Fit and Food which will result in more revenue over time.

## Data <a name="data"></a>

The data that is used for this project consistst of two parts. The first part is the NYC neighborhood data as obtained from course data by IBM.[1] In the case that the IBM Course link is not up anymore similar data can be found publicly available on the web.[2] The second part of the data comes from the FourSquare API, this data is freely available (with restrictions) for signed up accounts.[3] FourSquare's location data contains the location of most venues as well as their respective user ratings. Foursquare is the main provider of location data, for example Google Maps.

[1]: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
[2]: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas-NTA-/cpf4-rkhq
[3]: https://foursquare.com/

### Imports of libraries
In the section below all the necessary libraries are imported.

In [1]:
import numpy as np 
import pandas as pd
import json 

# Convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Library to handle requests
import requests

# Tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import sklearn packages
from sklearn.cluster import KMeans

# Map rendering library
import folium 

# Package used to obtain working directory paths
import os

# Save dataframes for later use
import pickle



print('Libraries have been imported.')

Libraries have been imported.


### NYC Neighborhood data

As mentioned before the NYC neighborhood's data is obtained from a link provided in the IBM Data Science course. 

In [2]:
url_nyc_json = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json'
data_nyc_json = requests.get(url_nyc_json).json()

print('Data downloaded!')

Data downloaded!


In [3]:
data_nyc_json 
neighborhood_data = data_nyc_json['features']
neighborhood_data[0]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhood_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use Geocoder to obtain the latitude and longitude for New York City.

In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_nyc = location.latitude
longitude_nyc = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_nyc, longitude_nyc))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create the base map for NYC  
On the base map the neighborhoods will be shown. These neighborhoods will later be combined with the data obtained from FoursQuare.

In [6]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude_nyc, longitude_nyc], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Foursquare location Data 
Below the data from FourSquare is obtained. At first the credentials have to defined in order to do the API requests. Afterwards  a for loop will be created to obtain the data Fitness and Restaurant Data for each neighborhood.

#### Define FourSquare credentials

In [7]:
CLIENT_ID = 'XS2RROXYD33QMK3R4XNSAGOKQ4HH4YJJEF3U3RASV20P3H4Z' # your Foursquare ID
CLIENT_SECRET = 'NLBBOB2XY3PKKOEBJALNWINDLW4MXMEL4GJZNWCMD4XBQWTW' # your Foursquare Secret
VERSION = '20201019' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XS2RROXYD33QMK3R4XNSAGOKQ4HH4YJJEF3U3RASV20P3H4Z
CLIENT_SECRET:NLBBOB2XY3PKKOEBJALNWINDLW4MXMEL4GJZNWCMD4XBQWTW


### Get requests for all Neighborhoods of NYC
In this section a get-request will be done for all gyms/fitness centers and restaurants in NYC for each neighborhood. Where all neighborhoods are obtained from the NYC dataset. In order to mitigate possible performance issues only 100 results are requested per neighborhood.

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
...,...,...,...,...
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631


In [9]:
#Range for forloop, needed to loop over all indices 
Range = neighborhoods.index

#Foursquare category id's
categoryId_restaurant = "4d4b7105d754a06374d81259"
categoryId_gym = "4bf58dd8d48988d175941735"

#Initialize response dfs
restaurants_df = pd.DataFrame()
gyms_df = pd.DataFrame()



In [10]:
# function that extracts the category of the venue from the foursquare data
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

A function is defined in order to retrieve data for all neighborhoods as this is done for both gyms as well as restaurants separately. The reason for this is that there are likely less gyms in an area so if the query was combined we'd get significantly lower amount of gyms respective to restaurants.

In [26]:
def get_venues(category_ids,Range,response_df,CLIENT_ID,VERSION,neighborhoods):
    LIMIT = 100
    radius = 1000
    neighborhood_latitude = neighborhoods["Latitude"]
    neighborhood_longitude = neighborhoods["Longitude"]
    neighborhood_name = neighborhoods["Neighborhood"]
    for i in Range:
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude[i], 
            neighborhood_longitude[i], 
            radius, 
            LIMIT,
            category_ids
            )
        
        results = requests.get(url).json()
        
        #Catch key error
        try:
            venues = results['response']['groups'][0]['items']
            nearby_venues = json_normalize(venues) # flatten JSON

            # filter columns
            filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']


            nearby_venues =nearby_venues.loc[:, filtered_columns]

            # filter the category for each row
            nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

            # clean columns
            nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.
                                     columns]
            nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
            nearby_venues["neighborhood"] = neighborhood_name[i]
            response_df = response_df.append(nearby_venues,ignore_index = True)
            #response_df["neighborhood"] = neighborhood_name
            
        except KeyError:       
            response_df = response_df.append(nearby_venues,ignore_index = True)
            #response_df["neighborhood"] = neighborhood_name
        
    return response_df
    

In [12]:
# Obtain the path to the directory in which data is saved
cwd = os.path.dirname(os.getcwd())
data_path = cwd + "\src\data\datasets"

In [92]:
# Dont query FoursQuare in the case data already exsists
restaurants = {}
gyms = {}
dataLoaded = False
try:
    with open(data_path +'/restaurants.pkl', 'rb') as f:
        restaurants_df = pickle.load(f)
    with open(data_path +'/gyms.pkl', 'rb') as f:
        gyms_df = pickle.load(f)
    print('Data loaded.')
    dataLoaded = True
except:
    pass

# When no data exists run function that gets data from FourSquare
if not dataLoaded:
    restaurants_df = get_venues(categoryId_restaurant,Range,restaurants_df,CLIENT_ID,VERSION,neighborhoods)
    gyms_df = get_venues(categoryId_gym,Range,gyms_df,CLIENT_ID,VERSION,neighborhoods)
    
    # Let's persists this in local file system
    with open(data_path +'/restaurants.pkl', 'wb') as f:
        pickle.dump(restaurants_df, f)
    with open(data_path +'/gyms.pkl', 'wb') as f:
        pickle.dump(gyms_df, f)


Data loaded.


In [93]:
gyms_df.head()

,name,categories,lat,lng,neighborhood
0,Boxingology LLC,Boxing Gym,40.898556,-73.838979,Wakefield
1,Mount St Michael's Academy (track),Track,40.898688,-73.840709,Wakefield
0,My Gym,Gym / Fitness Center,40.872681,-73.829400,Co-op City
1,24 Hour Fitness,Gym / Fitness Center,40.867818,-73.824984,Co-op City
2,bally's total fitness bartow ave,Gym / Fitness Center,40.867290,-73.832603,Co-op City


### Data manipulations

Several more manipulations to the data will be done before it can be analysed. Namely adding the general restaurant tag to the restaurants dataframe as well as the gym tag to the gyms dataframe. After this the data will be combined with the neighborhood data.

In [94]:
# Restaurant tag
restaurants_df["category"] = "Restaurant"
# Gym tag
gyms_df["category"] = "Gym"

In [95]:
#Add gym and restaurants together
df_overall = restaurants_df
df_overall = df_overall.append(gyms_df)
df_overall

,name,categories,lat,lng,neighborhood,category
0,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875,Wakefield,Restaurant
1,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310,Wakefield,Restaurant
2,Ali's Roti Shop,Caribbean Restaurant,40.894036,-73.856935,Wakefield,Restaurant
3,Jimbo's,Burger Joint,40.891740,-73.858226,Wakefield,Restaurant
4,Dunkin',Donut Shop,40.890459,-73.849089,Wakefield,Restaurant
...,...,...,...,...,...,...
31,Brooklyn Boulders Queensbridge,Climbing Gym,40.752649,-73.940010,Queensbridge,Gym
32,The Cliffs at Long Island City,Climbing Gym,40.748627,-73.948892,Queensbridge,Gym
0,Full Focus,Gym / Fitness Center,40.613092,-74.087140,Fox Hills,Gym
1,Zumba,Gym,40.624850,-74.081725,Fox Hills,Gym


#### One hot encoding
The one hot encoding technique is used to turn categorical values into a numeric representation.
Initially this is done for the neighborhoods and category.

In [96]:
#Get dummies for category
dummies_category = pd.get_dummies(df_overall["category"])
dummies_neighborhoods = pd.get_dummies(df_overall["neighborhood"])
df_dummy_neighborhood = pd.concat([df_overall,dummies_neighborhoods],axis = 1)
df_dummy_all = pd.concat([df_dummy_neighborhood,dummies_category],axis = 1)
neighborhoods_venues_dummied = df_dummy_all.copy()

## Methodology <a name="methodology"></a>

As a quick reminder: The data we have at this moment is: NYC neighborhood data, gyms in an area and the restaurants in an area.

The first part of the analysis will consists of an exploratory data analysis. Here the amount of restaurants and gyms in a neighborhoods will be evaluated. It is expected that insights will arise from this data about general neighborhoods that are crowded with restaurants/gyms or not. 

From the information that is obtained in the first part of the analysis additional features will be constructed. An example of this could be the ratio of Restaurants to gyms in a region or total gyms in a neighborhood. Another feature that could be used is the amount of fast food restaurants in a neighborhood which could be an indication of a non-healthy environment.

## Analysis <a name="analysis"></a>

The analysis will be done using clustering of neighborhoods. What we want to achieve with this clustering is defining neighborhoods dat are both healthy as well as not being to crowded with restaurants or gyms. In order to achieve this clustering the features that will be selected or created will be health and restaurant/fitness area-density related.

### Feature selection

### Group by neighborhood
For each neighborhood the following features are calculated:
* **Ratio of gyms to restaurants:** High ratio indicates a healthy region
* **Total amount of gyms:** A lot of gyms also mean healthier regions however we should strive for a healthy region with the lowest amount of gyms
* **Amount of restaurants:** We want to minimize the amount of restaurants


In [98]:
gym_mask = (neighborhoods_venues_dummied["category"] == "Gym")
restaurant_mask = (neighborhoods_venues_dummied["category"] == "Restaurant")
df = neighborhoods_venues_dummied
neighborhood_gym_count = df[df["category"]== "Gym" ].groupby('neighborhood').count()
neighborhood_restaurant_count = df[df["category"]== "Restaurant" ].groupby('neighborhood').count()

# Gym to restaurant ratio
neighborhood_gr_ratio = neighborhood_gym_count["category"]/neighborhood_restaurant_count['category']
gr_ratio = neighborhood_gr_ratio.reset_index()

# Total amount of gyms per Neighborhood
neighborhood_gym_count = neighborhood_gym_count["category"]
gym_count = neighborhood_gym_count.reset_index()

# Total amount of restaurants per Neighborhood
neighborhood_restaurant_count = neighborhood_restaurant_count['category'].reset_index()
restaurant_count = neighborhood_gym_count.reset_index()


In [99]:
neighborhoods_venues_dummied.reset_index(drop = True, inplace = True)

In [115]:
# Set proper datatypes and column names for merging

# Gym Restaurant Ratio
gr_ratio['neighborhood'] = gr_ratio['neighborhood'].astype("string")
gr_ratio.columns = ['neighborhood', 'gr_ratio']

# Gym count
gym_count['neighborhood'] = gym_count['neighborhood'].astype("string")
gym_count.columns = ['neighborhood', 'gym_count']

# Restaurant Count
restaurant_count['neighborhood'] = restaurant_count['neighborhood'].astype("string")
restaurant_count.columns = ['neighborhood', 'restaurant_count']





In [116]:
gr_ratio.dtypes

neighborhood     string
gr_ratio        float64
dtype: object

In [117]:
#Join values to the original dataframe

neighborhoods_venues = neighborhoods_venues_dummied.join(gr_ratio, on= 'neighborhood')
neighborhoods_venues = neighborhoods_venues.join(gym_count, on= 'neighborhood')
neighborhoods_venues = neighborhoods_venues.join(restaurant_count, on= 'neighborhood')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [101]:
neighborhoods_venues.head()

,name,categories,lat,lng,neighborhood,category,Allerton,Annadale,Arden Heights,Arlington,...,Woodlawn,Woodrow,Woodside,Yorkville,Gym,Restaurant,category,category,neighborhood,category
0,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875,Wakefield,Restaurant,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,Allerton,67.0
1,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310,Wakefield,Restaurant,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,Annadale,17.0
2,Ali's Roti Shop,Caribbean Restaurant,40.894036,-73.856935,Wakefield,Restaurant,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,Arden Heights,11.0
3,Jimbo's,Burger Joint,40.891740,-73.858226,Wakefield,Restaurant,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,Arlington,9.0
4,Dunkin',Donut Shop,40.890459,-73.849089,Wakefield,Restaurant,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,Arrochar,16.0


### Clustering

In [21]:
# Number of clusters corresponding to the amount of neighborhoods
k_clusters = 10

#Drop remaining categorical values from the dataframe
neighborhoods_grouped_clustering = df_dummy_all.drop(["category","categories","name","neighborhood"], axis = 1)

kmeans = KMeans(n_clusters = k_clusters).fit(neighborhoods_grouped_clustering)

# Check labels for the first 10 values
kmeans.labels_[0:10]

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [22]:
# add clustering labels
df_overall.insert(0, 'Cluster Labels', kmeans.labels_)


In [33]:
df_overall.tail()

,Cluster Labels,name,categories,lat,lng,neighborhood,category
23856,3,Brooklyn Boulders Queensbridge,Climbing Gym,40.752649,-73.940010,Queensbridge,Gym
23857,3,The Cliffs at Long Island City,Climbing Gym,40.748627,-73.948892,Queensbridge,Gym
23858,3,Full Focus,Gym / Fitness Center,40.613092,-74.087140,Fox Hills,Gym
23859,3,Zumba,Gym,40.624850,-74.081725,Fox Hills,Gym
23860,3,Relay For Life At Wagner,Track,40.611764,-74.074162,Fox Hills,Gym


In [34]:
#df_overall.reset_index(drop = True, inplace = True)
df_overall

,Cluster Labels,name,categories,lat,lng,neighborhood,category
0,9,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875,Wakefield,Restaurant
1,9,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310,Wakefield,Restaurant
2,9,Ali's Roti Shop,Caribbean Restaurant,40.894036,-73.856935,Wakefield,Restaurant
3,9,Jimbo's,Burger Joint,40.891740,-73.858226,Wakefield,Restaurant
4,9,Dunkin',Donut Shop,40.890459,-73.849089,Wakefield,Restaurant
...,...,...,...,...,...,...,...
23856,3,Brooklyn Boulders Queensbridge,Climbing Gym,40.752649,-73.940010,Queensbridge,Gym
23857,3,The Cliffs at Long Island City,Climbing Gym,40.748627,-73.948892,Queensbridge,Gym
23858,3,Full Focus,Gym / Fitness Center,40.613092,-74.087140,Fox Hills,Gym
23859,3,Zumba,Gym,40.624850,-74.081725,Fox Hills,Gym


## Results and Discussion <a name="results"></a>

### Use Geocoder to obtain the latitude and longitude for New York City.

In [24]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_nyc = location.latitude
longitude_nyc = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_nyc, longitude_nyc))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create the base map for NYC

In [25]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude_nyc, longitude_nyc], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Conclusion <a name="conclusion"></a>

## Recommendation <a name="recommendation"></a>

In order to improve the analysis done in this project several recommendations can be made. When comparing the amount of gyms in an area one could look at the population density in that neighborhood. This could be a possible explanation for a decreased number of gyms. Another improvement that could be made is , when possible to obtain, using public health data to cluster the neighborhoods. The reason for this is that a healthier neighborhood will more likely make use of Fit and Food. Furthermore during the development of the Fit and Food concept and franchise it is important to continiously re-evaluate the assessments done in this analysis based on changing circumstances or business findings.